## 0.Prepare folder directories

In [1]:
import os

SAMPLE_FOLDER = './sample/data/'
PAIRED_DESTINATION = ''

os.makedirs(os.path.join(SAMPLE_FOLDER, 'benchmarks'), exist_ok=True)
os.makedirs(os.path.join(SAMPLE_FOLDER, 'seqmaps'), exist_ok=True)
os.makedirs(os.path.join(SAMPLE_FOLDER, 'trackers'), exist_ok=True)

## 1. Import functions and specify configs

In [2]:
from utils.tracking import track_evaluate, create_sequence, create_benchmark,\
                    create_tracker_results, summarize, read_txt_to_mot

GT_PATH = './sample/gt.txt'
SEQUENCE_NAME = 'cam3_10mins'
FPS = 25
SHAPE = (1280,960)
LENGTH = 14952

BENCHMARK = 'topview'

# RESULT_PATH = './sample/bytetrack/bytetrack.txt'
RESULT_PATH = f'../byte_tracking/data/outputs/frcnn_r50_hdet640-cam3_1hour_1_byte.txt'

## 2. Run this cell in skip frame scenario

In [18]:
from utils.tracking import pair_gt_result

SKIP = 1
RESULT_PATH = f'./sample/bytetrack/bytetrack_{SKIP}skipframes.txt'
# RESULT_PATH = f'../byte_tracking/data/outputs/frcnn_r50_hdet640-cam3_1hour_1_byte_{SKIP}_skipframes.txt'
PAIRED_DESTINATION = './sample/paired_gt_result/'

paired = pair_gt_result(
    gt_path=GT_PATH, 
    result_path=RESULT_PATH,
    ref='gt',
    length=LENGTH,
    gt_frame_skip=SKIP,
    destination=PAIRED_DESTINATION,
    posfix='',
    verbose=True
) # -> paired: {'gt': gt, 'result': result, 'length': new_sequence_length}

new_sequence_length = paired['length']

LENGTH = new_sequence_length
GT_PATH = os.path.join(PAIRED_DESTINATION, os.path.basename(GT_PATH))
RESULT_PATH = os.path.join(PAIRED_DESTINATION, os.path.basename(RESULT_PATH))

Use 'gt' to align and reindex frame id
Truncate both file at frame id: 14952. Skip 1 frames and reindex to new length of: 7476
Paired groundtruth is created at: ./sample/paired_gt_result/gt.txt
Paired result is created at: ./sample/paired_gt_result/bytetrack_1skipframes.txt


## 3. Create the benchmark and results directories.

In [3]:
print('\n===== Create sequence folder', '='*5)
cam3_10mins = create_sequence(
    gt_path=GT_PATH,
    name=SEQUENCE_NAME,
    fps=FPS, shape=SHAPE, length=LENGTH,
    verbose=True
)

print('\n===== Create Benchmark folder', '='*5)
create_benchmark(
    benchmark_name=BENCHMARK, 
    sequences=[cam3_10mins], 
    destination=os.path.join(SAMPLE_FOLDER, 'benchmarks'),
    seqmap_path=os.path.join(SAMPLE_FOLDER, 'seqmaps'),
    verbose=True
)

print('\n===== Create result folders', '='*5)

byte_track_result_path = RESULT_PATH 
create_tracker_results(
    result_path=byte_track_result_path,
    benchmark_name=BENCHMARK, sequence_name=SEQUENCE_NAME, tracker_name='ByteTrack', 
    destination=os.path.join(SAMPLE_FOLDER, 'trackers'), 
    verbose=True
)


===== Create sequence folder =====
{'name': 'cam3_10mins', 'imDir': '', 'frameRate': 25, 'seqLength': 14952, 'imWidth': 1280, 'imHeight': 960, 'imExt': ''}

===== Create Benchmark folder =====
Benchmark 'topview' created at: ./sample/data/benchmarks/topview
Sequences: ['cam3_10mins']
Sequences mappings created at: ./sample/data/seqmaps/topview.txt

===== Create result folders =====
Result file of 'ByteTrack' for sequence 'cam3_10mins' of benchmark 'topview' is created at: ./sample/data/trackers/topview/ByteTrack/cam3_10mins.txt


## 4. Evaluation

In [4]:
gt_topview = os.path.join(SAMPLE_FOLDER, 'benchmarks', BENCHMARK)
tracker_topview = os.path.join(SAMPLE_FOLDER, 'trackers', BENCHMARK)
seqmap_file = os.path.join(SAMPLE_FOLDER, 'seqmaps', f'{BENCHMARK}.txt')


results = track_evaluate(benchmark=BENCHMARK, 
                         gt_folder=gt_topview, 
                         trackers_folder=tracker_topview, 
                         seqmap_file=seqmap_file)

summarize(results)


Evaluating 1 tracker(s) on 1 sequence(s) for 1 class(es) on MotChallenge2DBox dataset using the following metrics: HOTA, CLEAR, Identity, Count


Evaluating ByteTrack

==================== SUMMARY ====================
ByteTrack


,HOTA,MOTA,IDF1,IDsw
cam3_10mins,0.550697,0.77525,0.736598,20
COMBINED_SEQ,0.550697,0.77525,0.736598,20


<Figure size 432x288 with 0 Axes>

In [5]:
results

({'MotChallenge2DBox': {'ByteTrack': {'cam3_10mins': {'pedestrian': {'HOTA': {'HOTA': array([0.82619611, 0.82601942, 0.82569732, 0.82545003, 0.82522957,
              0.82432768, 0.81910619, 0.80693243, 0.78143044, 0.73983733,
              0.64408156, 0.54114661, 0.43471745, 0.32132194, 0.20362793,
              0.12390979, 0.06872222, 0.02117649, 0.00430333]),
       'DetA': array([9.24897857e-01, 9.24497708e-01, 9.23558012e-01, 9.22619233e-01,
              9.22120256e-01, 9.20624877e-01, 9.12550356e-01, 8.95108545e-01,
              8.54228632e-01, 7.89065822e-01, 6.42280605e-01, 4.81066121e-01,
              3.46623514e-01, 2.28332980e-01, 1.23152889e-01, 6.01188636e-02,
              2.34761507e-02, 4.61739635e-03, 4.21448486e-04]),
       'AssA': array([0.73802746, 0.73803113, 0.738206  , 0.73851458, 0.73851956,
              0.73810314, 0.7352306 , 0.72744243, 0.71483619, 0.69368012,
              0.64588757, 0.60873057, 0.54520036, 0.4521808 , 0.3366899 ,
              0.25538

## 5. Clean up if needed.

In [6]:
import shutil

shutil.rmtree(SAMPLE_FOLDER, ignore_errors=True)
shutil.rmtree(PAIRED_DESTINATION, ignore_errors=True)

In [7]:
! jupyter nbconvert --to markdown demo.ipynb

[NbConvertApp] Converting notebook demo.ipynb to markdown
[NbConvertApp] Writing 4086 bytes to demo.md
